### QC/trimming mcav 04142023


In [ ]:
#INSTALLATION
module load miniconda/22.11.1-1
conda create -n qc
conda activate qc
conda install -c bioconda trim-galore

In [ ]:
# edit past seq names to remove what the sequencer added
# in 04142023 dir: mcav 

#ex of filenames to edit:
# set2T1_12_2022_MCAV_S73_R1_001.fastq.gz

# working in dir: '/project/pi_sarah_gignouxwolfsohn_uml_edu/Raw_sequences/SCTLD_raw/04142023'
#copy mcav samples without '.md5' endings
ls *MCAV*.fastq.gz -1 | xargs -I {} cp {} /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav/raw

#move subset to new dir 
for file in *MCAV*.fastq.gz; do
    new_name=$(echo "$file" | sed 's/set2//;s/_S[0-9]*//')
    mv "$file" "$new_name"
done

FILEPATH='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav/raw'
OUTPUT_RESULTS='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav/trimmed/redo_01222023'

# create a file that gives all the unique filenames in the directory with the data, without the f/r specifiers
ls $FILEPATH -1 | sed 's/_R.*_001.fastq.gz//' | uniq | cat > $OUTPUT_RESULTS/'mcav_sampleids.txt'

#ensure all samples got copied over 
ls $FILEPATH -1 | wc -l # should be double:
wc -l mcav_sampleids.txt 

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=50G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 24:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/slurm-mcav_qc-%j.out  # %j = job ID

module load miniconda/22.11.1-1
conda activate qc

# Define the paths and variables
FILEPATH='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav/raw'
OUTPUT_RESULTS='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav/trimmed/redo_01222023'
NSLOTS=4  

SAMPLE_NAMES_FILE="/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/mcav/trimmed/redo_01222023/mcav_sampleids.txt"

# Check if the file exists
if [ ! -e "$SAMPLE_NAMES_FILE" ]; then
    echo "Error: $SAMPLE_NAMES_FILE does not exist."
    exit 1
fi

# Read each line from the file and perform actions
while IFS= read -r sample_id; do
    # Form the full file names
    input_r1="$FILEPATH/${sample_id}_R1_001.fastq.gz"
    input_r2="$FILEPATH/${sample_id}_R2_001.fastq.gz"
    
    # Ensure the input files exist before running the tools
    if [ ! -e "$input_r1" ] || [ ! -e "$input_r2" ]; then
        echo "Error: Input files do not exist for sample $sample_id"
        continue
    fi

    # Run trim_galore
    trim_galore -j "$NSLOTS" -q 20 --phred33 --illumina --length 20 --paired $input_r1 $input_r2 --fastqc -o $OUTPUT_RESULTS --dont_gzip


done < "$SAMPLE_NAMES_FILE"

# JOB-ID: 17302094
#trimmed read seqs in folder: 

### Fastqc info 

https://hbctraining.github.io/Intro-to-rnaseq-hpc-salmon/lessons/qc_fastqc_assessment.html

https://www.bioinformatics.babraham.ac.uk/projects/fastqc/